In [1]:
import numpy as np
import pandas as pd
from plotnine import *
from scipy.stats import binom, uniform

$\DeclareMathOperator*{\MEAN}{\mathbf{E}}$
$\DeclareMathOperator*{\VARIANCE}{\mathbf{D}}$
$\newcommand{\mean}[1]{\MEAN\left[#1\right]}$
$\newcommand{\variance}[1]{\VARIANCE\left[#1\right]}$
$\newcommand{\prob}[1]{\Pr\left[#1\right]}$
$\newcommand{\accuracy}{Acc}$
$\newcommand{\precision}{Prec}$
$\newcommand{\recall}{Rec}$

## Kvaliteedimõõtude vahede lähendid

Kuna defineeritud vahede täpseid väärtusi on praktikas keeruline leida hinnatakse neid kasutades valimi keskmisi. Sealjuures on loomulik mõlema meetodi hindamiseks kasutada sama valimit. Ühelt poolt on see standardpraktika -- tüüpiliselt hinnatakse masinõppemeetodite edukust fikseeritud testvalimite (\emph{benchmark}) peal. Teisalt tooks kahe valimi kasutamine vajaduse käsitsi märgendada rohkem andmepunkte ning lisab ka lõpptulemusse rohkem juhuslikkust.
Kui vastav testvalim sisaldab $N$ andmepunkti, siis saab õigsuste vahe lähendi avaldada järgnevalt
\begin{align}
    \Delta\widehat{\accuracy}&=\frac{1}{N}\cdot\sum_{i=1}^{N}[a_i=y_i]-\frac{1}{N}\cdot\sum_{i=1}^{N}[b_i=y_i] \nonumber\\
    &=\frac{1}{N}\cdot\sum_{i=1}^{N}[a_i=y_i]-[b_i=y_i]\nonumber\\
    &=\frac{1}{N}\cdot\sum_{a_i\neq b_i}[a_i=y_i]-[b_i=y_i] \enspace. \label{eq:õigsus vahe lähend}
\end{align}
Saadud võrduses \eqref{eq:õigsus vahe lähend} on summa märgi all nullist erinev arv parajasti siis, kui meetodite klassifikatsioonid on erinevad.
Sellest järeldub, et meetodite õigsuste vahe hindamiseks peab märgendama vaid andmepunkte, kus $a_i\neq b_i$.
Näiteks kui meetodid on üle $90\%$ õigsustega võivad nende klassifikatsioonid erineda maksimaalselt $20\%$ andmetest.
Sellisel juhul peab märgendama vaid iga viienda andmepunkti.
Veelgi enam, kuna iga summa liige valemis  \eqref{eq:õigsus vahe lähend} on $\pm 1$, saab hinnata $\Delta\widehat{\accuracy}$ ilma ühtegi andmepunkti märgendamata
\begin{equation}
    \label{eq:õigsus vahe lähend tõke}
    |\Delta\widehat{\accuracy}|\leq\frac{\#\{i: a_i\neq b_i\}}{N} \enspace,
\end{equation}
kus murru lugejas tähistab $\#$ hulga suurust.
Seega on võimalik veenduda, kas kaks algoritmi üldse õigsuse poolest erinevad andmepunktide tegelikke klasse teadmata.

Analoogselt õigsusele võib avaldada ka valimipõhiste täpsushinnagute vahe
\begin{equation}
    \label{eq:täpsus vahe lähend kole}
    \Delta\widehat{\precision}=\frac{\sum\limits_{i=1}^{N}[a_i=1]\cdot[y_i=1]}{\sum\limits_{i=1}^N[a_i=1]}-
    \frac{\sum\limits_{i=1}^{N}[b_i=1]\cdot[y_i=1]}{\sum\limits_{i=1}^N [b_i=1]} \enspace,
\end{equation}
mille edasise lihtsustamise muudab raskeks erinevus murru nimetajates.
Kuna tehniliselt ei pea täpsuse hidamisel kasutama sama testvalimit mõlema algoritmi jaoks, siis võib esialgset testvalmit kitsendada nii, et murru lugejad langevad kokku
\begin{equation*}
    \sum\limits_{i=1}^{N_A}[a_i=1]=S=\sum\limits_{i=1}^{N_B}[b_i=1] \enspace,
\end{equation*}
ning $N=\max(N_A, N_B)$.
Selline lähenemine on samaväärne valikumeetodi põhjal kahe $S$ andmepunktise valimi moodustamisega, kus vastuvõtutingimused on vastavalt $a_i=1$ ja $b_i=1$.
Seda arvestades saab valemi \eqref{eq:täpsus vahe lähend kole} viia lihtsustatud kujule
\begin{equation}
     \Delta\widehat{\precision}=\frac{1}{S}\cdot\sum\limits_{i=1}^{N_A}[a_i=1]\cdot[y_i=1]-\frac{1}{S}\cdot\sum\limits_{i=1}^{N_B}[b_i=1]\cdot[y_i=1] \enspace,
\end{equation}
kus $a_i$ on määratud vaid esimesel $N_A$ ja $b_i$ on määratud vaid esimesel $N_B$ andmepunktil.
Seega on $a_i$ ja $b_i$ väärtus kõigi $N$ punkti seas kas määramata, $0$ või $1$.
Siit lähtuvalt
\begin{align}
    \Delta\widehat{\precision}&=\frac{1}{S}\cdot\left(\sum_{\substack{a_i=1\\ b_i=1}}[y_i=1]+\sum_{\substack{a_i=1\\ b_i\neq 1}}[y_i=1]-\sum_{\substack{a_i=1\\ b_i=1}}[y_i=1]-\sum_{\substack{a_i\neq 1\\ b_i=1}}[y_i=1]\right) \nonumber\\
    &=\frac{1}{S}\cdot\left(\sum_{\substack{a_i=1\\ b_i\neq 1}}[y_i=1]-\sum_{\substack{a_i\neq 1\\ b_i=1}}[y_i=1]\right) \nonumber\\
    &=\frac{1}{S}\cdot\sum_{a_i\neq b_i}[a_i=1]\cdot[y_i=1]-[b_i=1]\cdot[y_i=1] \enspace. \label{eq:täpsus vahe lähend}
\end{align}
Andmepunktide märgendamise mõttes on täpsuse vahe lähend \eqref{eq:täpsus vahe lähend} samaväärne õigsuse valemiga \eqref{eq:õigsus vahe lähend}, sest hinnangu arvutamiseks peab märgendama vaid andmepunkte, mille puhul $a_i\neq b_i$.

Täpsuse vahehinnangu praktiline arvutamine algab $S$ fikseerimisest.
Seejärel tuleb valimile rakendada meetodeid kuni kumbki on $S$ andmepunkti positiivseks klassifitseerinud.
Märgendama peab valimi andmepunktid, mille puhul on olemas mõlema meetodi klassifikatsioonid, mis on üksteisest erinevad.
Nüüd on võimalik arvuta summa liikmete väärtused ning seejärel hinnang täpsuste vahele.
Sellega on oluliselt vähendadut märgendamist vajavate andmete hulka.

Saagise vahe avaldub sarnaselt täpsusele, lähtudes valemist
\begin{equation*}
    \Delta\widehat{\recall}=\frac{\sum\limits_{i=1}^{N}[a_i=1]\cdot[y_i=1]}{\sum\limits_{i=1}^{N}[y_i=1]}-\frac{\sum\limits_{i=1}^{N}[b_i=1]\cdot[y_i=1]}{\sum\limits_{i=1}^{N}[y_i=1]} \enspace,
\end{equation*}
kus erinevalt täpsusest on murdude nimetajad definitsiooni järgi võrdsed.
Tähistagu nimetajas olevat summat
\begin{equation*}
    T=\sum_{i=1}^N[y_i=1] \enspace.
\end{equation*}
Korrates täpsuse valemi \eqref{eq:täpsus vahe lähend} tuletamisega analoogset mõttekäiku, võib esitada saagise vahe hinnangu kujul
\begin{equation}
    \label{eq:saagis vahe lähend}
     \Delta\widehat{\recall}=\frac{1}{T}\cdot\sum_{a_i\neq b_i}[a_i=1]\cdot[y_i=1]-[b_i=1]\cdot[y_i=1] \enspace. 
\end{equation}
Märgendamise seisukohalt on $T$ arvutamine kulukas, kuna iga summa liikme puhul peab teadma andmepunkti tegelikku klassi.
Kõigi $N$ andmepunkti manuaalne märgendamine on väga resursimahukas. 
Alternatiiviks on väiksema valimi põhjal positiivse klassi esinemise sageduse ennustamine. 
See on võimalik vaid siis kui positiivse klassi esinemise sagedus on piisavalt suur.
Kui positiivse klassi esindajad on sagedusega $1:1000$, siis on tarvis adekvaatse hinnagu saamiseks läbi vaadata üle kümnetuhande andmepunkti. 
Seega on valemi \eqref{eq:saagis vahe lähend} praktiline rakendamine raskendatud veelgi madalama esinemisagedusega sündmuste korral.